# Nearest Neighbors Algorithm for Supervised Learning


Nearest neighbors algorithms (NNAs) are very simple conceptually: to classify a datum with specific feature values, find the data point that has the most similar feature values and put the original datum in that class. NNAs can also be used to predict missing feature values. 

The most common NNA is the k-Nearest Neighbors algorithm where the top *K* nearest neighbors to the query are identified. In most instantiations of k-NNA, classification/prediction is based on a "majority vote" of the k nearest neighbors (ex. if k = 5 and 3 out of the 5 nearest neighbors are class A and 2 are class B, the new data point will be classified as A). In the image below, using k = 1 would yield a class 1 classification while k = 3 would yield class 2.


![alt text](https://cdn-images-1.medium.com/max/1600/0*Sk18h9op6uK9EpT8.)


----

### A quick example to illustrate k-NNA:

If we want to classify a car as "cool" or "uncool" using the features *horsepower, number of seats,* and *manual (0) or automatic (1)*, our dataset might look like this:

*   150, 5, 0, uncool (2008 Honda Civic)  
*   320, 5, 0, cool (2011 Dodge Charger)
*   383, 3, 1, cool (1985 Chevy Blazer)
*   210, 7, 0, uncool (2001 Honda Odyssey)

Let's say we're trying to predict whether the 2017 Bugatti Veyron (1500hp, 2 seats, manual: 1) is cool or not. In the next cell the data is loaded. 





In [2]:
import numpy as np
import pandas as pd
import time
import math

In [2]:
cars_dict = {'2008 Honda Civic':    {'hp':150., 'seats':5., 'auto':0., 'cool':0}, 
             '2011 Dodge Charger' : {'hp':320., 'seats':5., 'auto':0., 'cool':1}, 
            '1985 Chevy Blazer':    {'hp':383., 'seats':3., 'auto':1., 'cool':1}, 
             '2001 Honda Odyssey':  {'hp':210., 'seats':7., 'auto':0., 'cool':0}, 
             '2017 Bugatti Veyron': {'hp':1500.,'seats':2., 'auto':1., 'cool':None}}

data = pd.DataFrame.from_dict(cars_dict,orient='index')
data

,hp,seats,auto,cool
1985 Chevy Blazer,383.0,3.0,1.0,1.0
2001 Honda Odyssey,210.0,7.0,0.0,0.0
2008 Honda Civic,150.0,5.0,0.0,0.0
2011 Dodge Charger,320.0,5.0,0.0,1.0
2017 Bugatti Veyron,1500.0,2.0,1.0,NaN


### Normalizing Data and Calculating Distance

In order to tell how "near" one datum is to another, we need a way to measure the distance between two data. One of the simplest ways to do this (and the way we will be doing it) is with simple Euclidian distance (formula below). Euclidean distance is the square root of the sum of the difference between each feature squared:

![alt text](https://i.stack.imgur.com/2y0bx.png.)

Some other approaches include Chi square distance and cosine distance (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4978658/). 

NOTE: By normalizing the data in this way we are assuming all features are equally important, but there are ways to weight some features more/less than others.

If we simply compute the Euclidean distance from one datum to another, values that are generally larger (like horsepower) will end up having a greater effect than the other smaller values. Because features with large values are not inherently more important for prediction than other values, we should normalize the data before we calculate distance. One quick and easy way to normalize data is to divide each datum by the maximum value in its category (i.e. divide each element in a row by the max value in that row).

In the cells below, fill in the code to normalize the data and create a function that will calculate the euclidean distance between two cars. 

In [3]:
# Normalize the data by dividing each value by the maximum value in its row. 
# Do not normalize the labels indicating cool and uncool (row 3)

for i in ['hp','seats','auto']:
    data[i] = data[i]/max(data[i].values)
  
data

,hp,seats,auto,cool
1985 Chevy Blazer,0.255333,0.428571,1.0,1.0
2001 Honda Odyssey,0.140000,1.000000,0.0,0.0
2008 Honda Civic,0.100000,0.714286,0.0,0.0
2011 Dodge Charger,0.213333,0.714286,0.0,1.0
2017 Bugatti Veyron,1.000000,0.285714,1.0,NaN


In [4]:
# Normalize data by removing the mean and scaling to unit variance from each feature

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_unitnorm = pd.DataFrame.from_dict(cars_dict,orient='index')

for i in ['hp','seats','auto']:
    feature_data = data_unitnorm[i].values.reshape(-1, 1)
    scaler.fit(feature_data)
    data_unitnorm[i] = scaler.transform(feature_data)
    
data_unitnorm

,hp,seats,auto,cool
1985 Chevy Blazer,-0.259004,-0.802955,1.224745,1.0
2001 Honda Odyssey,-0.604742,1.491202,-0.816497,0.0
2008 Honda Civic,-0.724651,0.344124,-0.816497,0.0
2011 Dodge Charger,-0.384908,0.344124,-0.816497,1.0
2017 Bugatti Veyron,1.973305,-1.376494,1.224745,NaN


In [5]:
# Distance Function using formula for euclidean distance

def euclidean_dist(new_datum, other_datum):

    inner_val = 0.0
    
    for g in range(new_datum.shape[0]):
        inner_val += (new_datum[g]- other_datum[g]) ** 2
    
    distance = math.sqrt(inner_val)
    return(distance)

In the cell below, calculate the distance between the Bugatti and each other car using euclidean_dist. Remember not to use row three (cool/uncool label) when computing distance.



In [6]:
# This is how you can call a specific row by name and sub-select features
bugatti = data.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values
bugatti

array([1.        , 0.28571429, 1.        ])

In [7]:
print('Euclidean Distances to 2017 Bugatti Veyron (V1)')
for car in list(data.index):
    d = euclidean_dist(bugatti, 
                       data.loc["{}".format(car)][["hp","seats","auto"]].values)
    print('  {}: \t{:01.3f}'.format(car,d))
    
print('\nEuclidean Distances to 2017 Bugatti Veyron (V2)')
for car in list(data_unitnorm.index):
    d = euclidean_dist(data_unitnorm.loc["2017 Bugatti Veyron"][["hp","seats","auto"]].values, 
                       data_unitnorm.loc["{}".format(car)][["hp","seats","auto"]].values)
    print('  {}: \t{:01.3f}'.format(car,d))

Euclidean Distances to 2017 Bugatti Veyron (V1)
  1985 Chevy Blazer: 	0.758
  2001 Honda Odyssey: 	1.500
  2008 Honda Civic: 	1.412
  2011 Dodge Charger: 	1.343
  2017 Bugatti Veyron: 	0.000

Euclidean Distances to 2017 Bugatti Veyron (V2)
  1985 Chevy Blazer: 	2.305
  2001 Honda Odyssey: 	4.363
  2008 Honda Civic: 	3.796
  2011 Dodge Charger: 	3.562
  2017 Bugatti Veyron: 	0.000


You should get the following distances (rounded to the thousandths palce):

Bugatti - Blazer = 0.758

Bugatti - Odyssey = 1.500

Bugatti - Civic = 1.412

Bugatti - Charger = 1.343

Because the distance between the Bugatti and Blazer is the smallest, if k = 1, we would classify the Bugatti as cool. If k = 4 we would not be able to classify the Bugatti in either category using the "majority vote" technique unless we had in place a tiebreaker protocol. Generally speaking, larger values of *k* reduce noise, but also make the boundaries between classes less distinct. The best value of *k* will depend on your dataset and your prediction needs.

# Predicting Diabetes in Pima Heritage Dataset

Next we will see if we can use k-NNA to predict whether or not a patient has diabetes given some medical information about them. Load and view the data in the cell below. 




In [3]:
# Data Loading and preprocessing

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

#  'preg': number of pregnancies  
#  'plas': plasma glucose concentration 
#  'pres': blood pressure 
#  'skin': skin thickness
#  'test': Insulin
#  'mass': BMI
#  'pedi': diabetes pedigree function
#  'age': age
#  'class': '0' means does not have diabetes and '1' means has diabetes

data.head()


,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Now, let's clearly define which columns will act as explanatory variables, and which column will be the target value, and split the dataset between your training data and testing data.

In [4]:
from sklearn.model_selection import train_test_split

# columns we will use to make predictions with
x_cols = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']

# column that we want to predict
y_col = 'class'

# 80-20 split of dataset
test_size = 0.2
x_training, x_testing, y_training, y_testing = train_test_split(data[x_cols], data[y_col], test_size=test_size, random_state=0)

print(x_training.shape, y_training.shape)
print(x_testing.shape, y_testing.shape)

(614, 8) (614,)
(154, 8) (154,)


Let's not forget to normalize the data!

In [5]:
scaler = StandardScaler()

for i in list(x_training):
    feature_data_train = x_training[i].values.reshape(-1, 1)
    feature_data_test = x_testing[i].values.reshape(-1, 1)
    
    # normalize only according to training data!
    scaler.fit(feature_data_train)
    
    x_training[i] = scaler.transform(feature_data_train)
    x_testing[i] = scaler.transform(feature_data_test)
    


NameError: name 'StandardScaler' is not defined

This predict method needs to compute the euclidean distance between the “new” observation and all the data points in the training set. Then, it assigns the corresponding label to the observation. Finally, it selects the K nearest ones and performs a "majority vote."



In [11]:
from collections import Counter

def predict(x_training, y_training, x_test_sample, k):
    
    # create list for distances and targets
    distances = []
    targets = []

    # use our previously made euclidean distance function, calculate distance to all samples in training set
    for q in list(x_training.index):
        distances.append([euclidean_dist(x_test_sample, x_training.loc[q]), q])
  
    # sort distances (smallest to largest)
    distances = sorted(distances)
  
    # make a list of the k neighbors' targets
    for i in range(k):
        dix = distances[i][1]
        targets.append(y_training.loc[dix])
        
    # retrieve the most common target
    c=Counter(targets)
    return c.most_common()[0][0]


In [12]:
def knn(x_training, y_training, x_testing, k):
    
    predictions = np.empty([x_testing.shape[0],])
    
    # loop over all test samples
    px = 0
    for i in list(x_testing.index):
        predictions[px]=predict(x_training, y_training, x_testing.loc[i], k)
        px+=1
        
    return predictions.astype(int)

In [13]:
start = time.time()
predictions_slow = knn(x_training, y_training, x_testing, k=5)

print('Took {} seconds'.format(time.time() - start))
predictions_slow

Took 48.11781144142151 seconds


array([1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Using sklearn to speed up normalizing and distance finding

Luckily for us, sklearn has some quick and easy functions for normalizing the data, finding Euclidean distances, training, and testing with k-NNA. Try k = 5 to start.

In [14]:
from sklearn.neighbors import KNeighborsClassifier

# Create Model with k nearest neighbors
knn = KNeighborsClassifier(n_neighbors=5)

# Train/fit model with training data
knn.fit(x_training, y_training)

# Make predictions on the test data using the fitted model
start = time.time()
predictions_fast = knn.predict(x_testing)

print('Took {} seconds'.format(time.time() - start))
predictions_fast

Took 0.003000497817993164 seconds


array([1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

### Evaluating classification performance
Let's see how well our classifier did!

In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

cm = confusion_matrix(y_testing, predictions_fast)

print('Confusion Matrix: \n',cm)
print('\nF1: ',f1_score(y_testing, predictions_fast,labels=[0,1]))

tn, fp, fn, tp = cm.ravel()
print('\nTN: ',tn, '\nFP: ',fp, '\nFN: ',fn, '\nTP: ',tp)

Confusion Matrix: 
 [[93 14]
 [17 30]]

F1:  0.6593406593406593

TN:  93 
FP:  14 
FN:  17 
TP:  30


You should get F1 = 0.659. Notice that this classifier is pretty good at classifying negative samples (compare TN to FP), but we are not very good at classifying positive samples (compare TP to FN).

Spend a few minutes trying to increase the F1 score as much as you can by changing k and the features of the data you are using to predict values.

* What set of features and values of k did you find to be the most optimal?
* Why is choosing the right features so important for prediction accuracy? 
* What other model hyperparameters (other than k) might we be able to tune?
* How might we use non-numerical data columns in our model (if we had any)?

# Pros and Cons of k-NNA

## Pros
* Non-parametric (can be used with data that does not fit a normal distribution)
* Conceptually simple and relatively simple to instantiate
* Little to no "training" time 
* A good starting point/baseline classifier 

## Cons
* Slow "testing" phase compared to other predictors/classifiers 
* Degrades with high-dimension data (because there is less difference between closest and furthest neighbors)
* Unclear how to handle non-numeric features
* Doesn't handle data with skewed class distribution well (if one class is extremely dominant in the training data, it will dominate the "voting majority" for classifying new data)
* Features that are not of the same scale can (should) be normalized, but this introduces another model hyperparameter (i.e., which normalization method is appropriate?)
* Does not take into account feature correlation

---



From these exercises and the pros and cons listed above, here is when it is most useful to use k-NNA:
* Datasets with many data points and few dimensions (but can become very slow as well)
* Datasets that are non-parametric
* When you want a quick and easy classifier that does not have to be optimal (perhaps to use as a baseline for other models)


Lastly, there are many other types of NNAs and there are many other ways to instantiate the k-NNA. For example, we could have used a weighted voting system (nearer neighbors' votes carry more weight) instead of a majority voting system to classify. Additionally, we could have used a different distance measurement or a different error measurement. 